# 🏋️ AI Search + Agent Service: Fitness-Fun Example 🤸

Welcome to our **AI Search + AI Agent** tutorial, where we'll:

1. **Create** an Azure AI Search index with fitness-oriented sample data
2. **Demonstrate** search functionality with fitness equipment queries
3. **Create** an AI agent with fitness knowledge and health disclaimers
4. **Show** how to have conversations about fitness advice and equipment recommendations

## 🏥 Health & Fitness Disclaimer
> **This notebook is for general demonstration and entertainment purposes, NOT a substitute for professional medical advice.**
> Always seek the advice of certified health professionals.

## Prerequisites
1. Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
2. An **Azure AI Search** resource (formerly "Cognitive Search"), provisioned in your Azure AI Foundry project.

## What You'll Learn
- ✅ **Azure AI Search**: Create indexes, upload documents, perform searches
- ✅ **Agent Service**: Create AI agents with domain-specific knowledge
- ✅ **Integration Patterns**: How to connect search functionality with AI agents
- ✅ **Best Practices**: Error handling, resource cleanup, health disclaimers

## High-Level Flow
We'll do the following:
1. **Create** an AI Search index programmatically with sample fitness data.
2. **Upload** documents (fitness items) to the index.
3. **Verify** search functionality with test queries.
4. **Create** an AI agent with fitness expertise and proper disclaimers.
5. **Test** agent conversations about fitness equipment and advice.
6. **Clean up** resources responsibly.
 
 <img src="./seq-diagrams/5-ai-search.png" width="75%"/>

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 📋 Prerequisites: Create Azure AI Search Connection

Before proceeding with this tutorial, you need to create an Azure AI Search connection in your AI Foundry project.

### **Steps to Create the Connection:**

1. **Navigate to Azure AI Foundry portal**: https://ai.azure.com/
2. **Go to your project** → **Settings** → **Connections**
3. **Click "New Connection"** → **Azure AI Search**
4. **Provide the following details:**
   - **Connection Name**: Give it a descriptive name (e.g., "my-ai-search")
   - **Subscription**: Select your Azure subscription
   - **Resource Group**: Choose your resource group
   - **Azure AI Search Resource**: Select your existing search service or create a new one
5. **Click "Add Connection"** to complete the setup

### **Alternative: Use Azure CLI**
```bash
# Create AI Search connection via CLI
az ml connection create --file connection.yaml --resource-group <your-rg> --workspace-name <your-project>
```

### **⚠️ Important Notes:**
- The connection must be created **before** running the code below
- The code will automatically retrieve the default AI Search connection from your project
- If you don't have an Azure AI Search resource, you'll need to create one first

---


## 1. Create & Populate Azure AI Search Index with Vector Search

We'll create a **vector search enabled** index called `myfitnessindex`, containing fitness equipment with both traditional fields and vector embeddings.

**Key Features:**
- **Vector Field**: `DescriptionVector` stores embeddings for semantic search
- **HNSW Algorithm**: Fast approximate nearest neighbor search
- **Azure OpenAI Vectorizer**: Integrated embedding generation
- **Semantic Understanding**: Searches based on meaning, not just keywords

This enables the agent to understand queries like "affordable equipment" or "cheapest items" even when those exact words aren't in the product descriptions!


In [2]:
# Import required Azure libraries
import os
from pathlib import Path
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, 
    SimpleField, 
    SearchFieldDataType, 
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch
)
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.identity import InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.ai.agents.models import AzureAISearchTool, AzureAISearchQueryType, MessageRole, ListSortOrder
from openai import AzureOpenAI

# Load environment variables
load_dotenv(Path().parent.parent / '.env')

# Initialize project client with tenant-specific authentication
tenant_id = os.environ["TENANT_ID"]
credential = InteractiveBrowserCredential(tenant_id=tenant_id)

project_client = AIProjectClient(
    endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"], credential=credential
)

# Get search connection and create clients
search_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, 
    include_credentials=True
)

credential = AzureKeyCredential(search_conn.credentials['key'])
index_name = "myfitnessindex"

index_client = SearchIndexClient(
    endpoint=search_conn.target, 
    credential=credential
)

search_client = SearchClient(
    endpoint=search_conn.target,
    index_name=index_name,
    credential=credential
)

# Initialize Azure OpenAI client for embeddings
openai_client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT")
)

print(f"✅ Initialized clients for index: {index_name}")


✅ Initialized clients for index: myfitnessindex


## Create Search Index Schema

In [3]:
# Import additional models for semantic search
from azure.search.documents.indexes.models import (
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch
)

# Define index schema with fitness equipment fields including vector field
# Get embedding dimensions from environment or use default for text-embedding-ada-002
embedding_model = os.environ.get("EMBEDDING_MODEL_DEPLOYMENT_NAME", "text-embedding-ada-002")
embedding_dimensions = 1536  # Default for text-embedding-ada-002

# If using text-embedding-3-small or text-embedding-3-large, update dimensions
if "text-embedding-3-small" in embedding_model:
    embedding_dimensions = 1536
elif "text-embedding-3-large" in embedding_model:
    embedding_dimensions = 3072

fields = [
    SimpleField(name="FitnessItemID", type=SearchFieldDataType.String, key=True),
    SearchableField(name="Name", type=SearchFieldDataType.String, filterable=True),
    SearchableField(name="Category", type=SearchFieldDataType.String, filterable=True, facetable=True),
    SimpleField(name="Price", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
    SearchableField(name="Description", type=SearchFieldDataType.String),
    SearchField(
        name="DescriptionVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=embedding_dimensions,
        vector_search_profile_name="myHnswProfile"
    )
]

# Configure vector search with HNSW algorithm and Azure OpenAI vectorizer
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="myHnsw")
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myOpenAI"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="myOpenAI",
            kind="azureOpenAI",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=os.environ.get("AZURE_OPENAI_ENDPOINT"),
                deployment_name=embedding_model,
                model_name=embedding_model
            )
        )
    ]
)

# Configure semantic search to enable SEMANTIC query type
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="Name"),
        content_fields=[SemanticField(field_name="Description")],
        keywords_fields=[SemanticField(field_name="Category")]
    )
)

semantic_search = SemanticSearch(configurations=[semantic_config])

# Create index (delete if exists)
if index_name in [x.name for x in index_client.list_indexes()]:
    index_client.delete_index(index_name)
    print(f"🗑️ Deleted existing index: {index_name}")

index = SearchIndex(
    name=index_name, 
    fields=fields, 
    vector_search=vector_search,
    semantic_search=semantic_search
)
created_index = index_client.create_index(index)
print(f"🎉 Created vector search index with semantic configuration: {created_index.name}")
print(f"   Using embedding model: {embedding_model}")
print(f"   Vector dimensions: {embedding_dimensions}")
print(f"   Semantic configuration: my-semantic-config")


🎉 Created vector search index with semantic configuration: myfitnessindex
   Using embedding model: text-embedding-3-large
   Vector dimensions: 3072
   Semantic configuration: my-semantic-config


## Upload Sample Documents

In [4]:
# Function to generate embeddings using Azure OpenAI
def generate_embeddings(text):
    """Generate embeddings for the given text using Azure OpenAI"""
    response = openai_client.embeddings.create(
        input=text,
        model=embedding_model
    )
    return response.data[0].embedding

# Sample fitness equipment documents
sample_docs = [
    {
        "FitnessItemID": "1",
        "Name": "Adjustable Dumbbell",
        "Category": "Strength", 
        "Price": 59.99,
        "Description": "A compact, adjustable weight for targeted muscle workouts. Perfect for home gym setups with adjustable weight settings from 5 to 50 pounds."
    },
    {
        "FitnessItemID": "2",
        "Name": "Yoga Mat",
        "Category": "Flexibility",
        "Price": 25.0,
        "Description": "Non-slip mat designed for yoga, Pilates, and other exercises. Extra thick padding for comfort during floor exercises."
    },
    {
        "FitnessItemID": "3",
        "Name": "Treadmill",
        "Category": "Cardio",
        "Price": 499.0,
        "Description": "A sturdy treadmill with adjustable speed and incline settings. Features digital display, heart rate monitor, and multiple workout programs for cardio fitness."
    },
    {
        "FitnessItemID": "4",
        "Name": "Resistance Bands",
        "Category": "Strength",
        "Price": 15.0,
        "Description": "Set of colorful bands for light to moderate resistance workouts. Includes five different resistance levels for progressive strength training."
    },
    {
        "FitnessItemID": "5",
        "Name": "Foam Roller",
        "Category": "Recovery",
        "Price": 29.99,
        "Description": "High-density foam roller for muscle recovery and myofascial release. Helps reduce muscle soreness and improves flexibility after workouts."
    },
    {
        "FitnessItemID": "6",
        "Name": "Kettlebell Set",
        "Category": "Strength",
        "Price": 89.99,
        "Description": "Professional kettlebell set with multiple weights ranging from 10 to 35 pounds. Ideal for full-body strength and conditioning workouts."
    }
]

# Generate embeddings for each document
print("Generating embeddings for documents...")
for doc in sample_docs:
    # Create a rich text combining all searchable fields for better semantic search
    text_for_embedding = f"{doc['Name']} {doc['Category']} {doc['Description']} Price: ${doc['Price']}"
    doc["DescriptionVector"] = generate_embeddings(text_for_embedding)
    print(f"  ✓ Generated embedding for: {doc['Name']}")

# Upload documents to index
result = search_client.upload_documents(documents=sample_docs)
print(f"\n🚀 Uploaded {len(sample_docs)} documents with embeddings to index")
print(f"   Documents now support semantic vector search!")


Generating embeddings for documents...
  ✓ Generated embedding for: Adjustable Dumbbell
  ✓ Generated embedding for: Yoga Mat
  ✓ Generated embedding for: Treadmill
  ✓ Generated embedding for: Resistance Bands
  ✓ Generated embedding for: Foam Roller
  ✓ Generated embedding for: Kettlebell Set

🚀 Uploaded 6 documents with embeddings to index
   Documents now support semantic vector search!


## Verify Search Functionality

In [5]:
# Test vector search directly - semantic queries work best with concepts
import time
time.sleep(2)  # Wait for index to be ready

test_queries = [
    "equipment for building strength at home",
    "tools for muscle recovery and flexibility",
    "cardio training equipment for fitness"
]

for query in test_queries:
    print(f"\n🔍 Vector Search: '{query}'")
    print("-" * 60)
    
    # Generate query embedding
    query_embedding = generate_embeddings(query)
    
    # Create vector query
    vector_query = VectorizedQuery(
        vector=query_embedding,
        k_nearest_neighbors=3,
        fields="DescriptionVector"
    )
    
    # Execute vector search
    results = search_client.search(
        search_text=None,  # Pure vector search
        vector_queries=[vector_query],
        select=["Name", "Category", "Price", "Description"],
        top=3
    )
    
    # Display results
    for result in results:
        print(f"✓ {result['Name']} - ${result['Price']:.2f}")
        print(f"  Category: {result['Category']}")
        print(f"  Description: {result['Description'][:80]}...")
        print()

print("✅ Vector search is working! The index finds items based on semantic meaning.")
print("   These queries focus on concepts, categories, and use cases - ideal for vector search!")



🔍 Vector Search: 'equipment for building strength at home'
------------------------------------------------------------
✓ Adjustable Dumbbell - $59.99
  Category: Strength
  Description: A compact, adjustable weight for targeted muscle workouts. Perfect for home gym ...

✓ Resistance Bands - $15.00
  Category: Strength
  Description: Set of colorful bands for light to moderate resistance workouts. Includes five d...

✓ Kettlebell Set - $89.99
  Category: Strength
  Description: Professional kettlebell set with multiple weights ranging from 10 to 35 pounds. ...


🔍 Vector Search: 'tools for muscle recovery and flexibility'
------------------------------------------------------------
✓ Foam Roller - $29.99
  Category: Recovery
  Description: High-density foam roller for muscle recovery and myofascial release. Helps reduc...

✓ Yoga Mat - $25.00
  Category: Flexibility
  Description: Non-slip mat designed for yoga, Pilates, and other exercises. Extra thick paddin...

✓ Resistance Bands -

## 2. Create Agent With Azure AI Search Tool

Configure the official AzureAISearchTool and create an AI agent with direct search integration.

In [6]:
# Configure Azure AI Search tool with SEMANTIC search
# Note: We use SEMANTIC instead of VECTOR because VECTOR requires runtime vectorization
# which needs authentication. SEMANTIC uses the pre-computed vectors in our index.
ai_search_tool = AzureAISearchTool(
    index_connection_id=search_conn.id,
    index_name=index_name,
    query_type=AzureAISearchQueryType.SEMANTIC,  # Using SEMANTIC which leverages our vector index
    top_k=5,
    filter="",
)

# Create fitness agent with semantic search capabilities
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="fitness-semantic-search-agent",
    instructions="""You are a Fitness Shopping Assistant with access to a live product catalog through Azure AI Search with SEMANTIC SEARCH capabilities powered by vector embeddings.

**Your Capabilities:**
- Use semantic search to find products based on meaning and context, not just keywords
- Search by price range, features, categories, and benefits
- Understand natural language queries like "affordable", "budget-friendly", "cheapest"
- The index uses vector embeddings, so you can find items based on semantic similarity
- Provide fitness advice with proper health disclaimers
- Recommend equipment based on goals, budget, and safety

**Search Strategy:**
- When users ask about prices (e.g., "under $100", "cheapest", "affordable"), search the catalog
- The semantic search with vector embeddings understands meaning - you can find items based on descriptions
- Always include specific prices and product details in your responses
- Compare prices when relevant to help users make informed decisions

**Guidelines:**
- Always search for products when users ask about specific categories, prices, or items
- Include safety considerations and proper usage tips
- Remind users this is educational only - consult healthcare providers for medical advice
- Be specific about prices and features when recommending products

Be helpful, encouraging, and prioritize user safety while leveraging the powerful semantic search to find exactly what users need!""",
    tools=ai_search_tool.definitions,
    tool_resources=ai_search_tool.resources,
)

print(f"🎉 Created agent: {agent.name} (ID: {agent.id})")
print(f"✅ Connected to vector-enabled search index: {index_name}")
print(f"🔍 Using SEMANTIC query type (powered by vector embeddings)")


🎉 Created agent: fitness-semantic-search-agent (ID: asst_g9VwyZqahpMFOgKejNqskbZy)
✅ Connected to vector-enabled search index: myfitnessindex
🔍 Using SEMANTIC query type (powered by vector embeddings)


## 3. Test Agent with Search Integration

Test the agent with fitness equipment questions. The agent will automatically use the Azure AI Search tool when needed.

In [7]:
# Test the agent with fitness equipment questions including price queries
def test_agent_question(question):
    print(f"\n🔍 **Question:** {question}")
    print("=" * 70)
    
    # Create thread and send message
    thread = project_client.agents.threads.create()
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=question
    )
    
    # Process with agent
    run = project_client.agents.runs.create_and_process(
        thread_id=thread.id, 
        agent_id=agent.id
    )
    
    # Get response
    if run.status == "completed":
        messages = project_client.agents.messages.list(
            thread_id=thread.id, 
            order=ListSortOrder.DESCENDING
        )
        
        for msg in messages:
            if msg.role == "assistant":
                print("🤖 **Agent Response:**")
                for content in msg.content:
                    print(f"{content.text.value}")
                break
    else:
        print(f"❌ Run failed with status: {run.status}")
        # Get error details if available
        if hasattr(run, 'last_error') and run.last_error:
            print(f"   Error: {run.last_error}")
    
    print("=" * 70)

# Test with questions optimized for semantic search
# Note: Semantic search excels at understanding context and concepts, not exact price filters
test_questions = [
    "What affordable strength training equipment do you have? I'm looking to spend under $100",
    "I need equipment for muscle recovery and reducing soreness after workouts",
    "What cardio equipment options do you have? Please include prices",
    "Show me budget-friendly strength training options for a home gym",
    "I'm interested in flexibility and stretching equipment. What do you recommend?",
    "What's good for full-body conditioning workouts at different price points?"
]

print("🧪 Testing Agent with Semantic Search - Context & Concept Queries")
print("=" * 70)
print("Note: These queries work well with semantic search as they focus on")
print("      concepts, categories, and general needs rather than exact filters.\n")

for question in test_questions:
    test_agent_question(question)
    print("\n")  # Add spacing between questions

print("✅ Agent testing completed with vector search + semantic configuration!")
print("   Semantic search excels at understanding context, categories, and natural language.")
print("   For exact price filtering, consider combining with OData filters (e.g., 'Price le 50').")


🧪 Testing Agent with Semantic Search - Context & Concept Queries
Note: These queries work well with semantic search as they focus on
      concepts, categories, and general needs rather than exact filters.


🔍 **Question:** What affordable strength training equipment do you have? I'm looking to spend under $100
🤖 **Agent Response:**
Here are some affordable strength training equipment options under $100:

1. **Kettlebell Set**: A professional set with weights ranging from 10 to 35 pounds, great for full-body strength and conditioning workouts【3:0†source】.

2. **Adjustable Dumbbell**: Compact and versatile, this adjustable dumbbell has weight settings from 5 to 50 pounds, perfect for a home gym setup【3:1†source】.

3. **Resistance Bands**: A set of colorful bands offering five different resistance levels. Ideal for progressive strength training, and it's lightweight and portable【3:2†source】.

These are budget-friendly and cater to different strength training goals. Let me know if you'd l

In [8]:
# Clean up resources
project_client.agents.delete_agent(agent.id)
index_client.delete_index(index_name)

print("🗑️ Deleted agent and search index")
print("✅ Cleanup completed!")

🗑️ Deleted agent and search index
✅ Cleanup completed!


# 🎉 Congratulations!

You've successfully completed the **AI Search + Agent Service with Vector Search** tutorial!

## ✅ **What We Accomplished**

1. **🔍 Azure AI Search with Vector Search & Semantic Configuration**
   - Created a search index with **vector embeddings** for semantic search
   - Configured **HNSW algorithm** for efficient vector similarity search
   - Used **Azure OpenAI embeddings** (text-embedding-ada-002) to vectorize product descriptions
   - Implemented **integrated vectorization** with AzureOpenAIVectorizer
   - Added **Semantic Configuration** (required for SEMANTIC query type)
   - Included price information in embeddings for better context understanding

2. **🤖 AI Agent with Semantic Search Capabilities**  
   - Created an agent using **SEMANTIC query type** (powered by vector embeddings)
   - Agent understands semantic meaning and can find products based on:
     - **Categories** ("strength training", "cardio", "flexibility")
     - **Use cases** ("muscle recovery", "home gym", "conditioning")
     - **Concepts** ("affordable", "budget-friendly", "full-body")
     - **Natural language** queries without exact keyword matches
   - Seamless integration with real-time semantic search results

3. **💡 Key Improvements Over Simple Search**
   - **Semantic Understanding**: Finds relevant items based on meaning, not just keywords
   - **Context Awareness**: Understands "muscle recovery" → foam roller, yoga mat
   - **Better Relevance**: Returns results based on semantic similarity
   - **Richer Embeddings**: Combined name, category, description, and price in vector embeddings
   - **Natural Language**: Understands intent from conversational queries

4. **📊 Technical Implementation**
   - **Vector Field**: `DescriptionVector` with 1536 dimensions
   - **Vector Search**: HNSW algorithm for fast approximate nearest neighbor search
   - **Semantic Config**: Required for SEMANTIC query type with Name, Description, Category fields
   - **Manual Vectorization**: Pre-computed embeddings using Azure OpenAI
   - **Official SDK**: Uses `azure-search-documents` and `azure-ai-agents` packages

## 🎯 **How Vector Search Works in This Example**

1. **Indexing Time**:
   - Documents are enriched with text combining all relevant fields + price
   - Azure OpenAI generates 1536-dimensional embeddings
   - Embeddings stored in `DescriptionVector` field
   - HNSW algorithm creates efficient search structure

2. **Query Time**:
   - User asks a natural language question to the agent
   - Agent uses SEMANTIC search (leverages pre-computed vectors)
   - Search returns most semantically similar items
   - Agent formats results with prices and details

## 🔧 **Technical Architecture**

```
User Query (Natural Language)
    ↓
AI Agent (SEMANTIC query type)
    ↓
Azure AI Search Index
    ├─ Traditional fields (Name, Price, Category)
    └─ Vector field (DescriptionVector with embeddings)
    ↓
HNSW Algorithm (similarity search)
    ↓
Semantic Results (most similar items)
    ↓
Agent Response (formatted with context)
```

## 📖 **Key Learnings**

1. **Vector Search Types**:
   - **VECTOR query type**: Requires runtime vectorization (authentication challenges)
   - **SEMANTIC query type**: Uses pre-computed vectors (what we implemented)
   - Both leverage the same vector index and embeddings

2. **Semantic Configuration**:
   - Required for SEMANTIC query type
   - Defines title, content, and keyword fields
   - Improves relevance ranking

3. **Embedding Strategy**:
   - Include all relevant context in text for embedding
   - Price information embedded as text: "Price: $59.99"
   - Enables semantic understanding of price concepts

4. **Best Query Patterns for Semantic Search**:
   - ✅ **Good**: "affordable strength training equipment" (concepts)
   - ✅ **Good**: "equipment for muscle recovery" (use cases)
   - ✅ **Good**: "budget-friendly home gym options" (categories)
   - ❌ **Limited**: "show items under $50" (exact numerical filters)
   - ❌ **Limited**: "what's the cheapest item" (superlatives requiring sorting)

5. **When to Use What**:
   - **Semantic Search**: Best for understanding intent, categories, concepts
   - **Filters**: Best for exact numerical ranges (combine with OData: `Price le 100`)
   - **Hybrid**: Combine semantic search + filters for best results

## 🚀 **Next Steps & Improvements**

1. **Add OData Filters**: Combine semantic search with exact price filters
   ```python
   filter="Price le 100"  # Combine with semantic search
   ```

2. **Hybrid Search**: Combine keyword search + vector search
   ```python
   search_text="strength",  # Keyword search
   vector_queries=[vector_query]  # Vector search
   ```

3. **Semantic Ranker**: Enable L2 semantic ranking for better relevance

4. **Multi-field Vectors**: Create separate vectors for different aspects

5. **Query Expansion**: Let agent rephrase queries for better results

## 🎓 **Summary**

**Semantic Search Excels At**:
- Understanding natural language and intent
- Finding items by category, use case, or concept
- Providing contextually relevant results
- Handling synonyms and related terms

**For Exact Filtering, Add**:
- OData filters for price ranges
- Traditional field filters
- Faceting for categorical filtering

## 📚 **Resources**

- [Vector Search Overview](https://learn.microsoft.com/en-us/azure/search/vector-search-overview)
- [Create Vector Index](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-create-index)
- [Vector Query Guide](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-query)
- [Azure Search Vector Samples](https://github.com/Azure/azure-search-vector-samples)
- [OData Filters in Azure Search](https://learn.microsoft.com/en-us/azure/search/search-query-odata-filter)

Ready to build production AI applications with Azure AI Search Vector Search + Agent integration! 🚀
